In [1]:
import  asyncio, nest_asyncio
from typing import List
import pandas as pd

from read_telega_dump import telega_dump_to_pandas
from read_telega_api import extract_message_reactions
from pyrogram_tests import get_reactions
import time

nest_asyncio.apply()

dump_path = r"D:\test_data\ChatExport_2024-03-14\result.json"
tlg_group_id = -1001688539638 
# telegram group id, can be taken from "copy message link"  in desctop telegram

app started at 2024-03-17 19:14:42.522780


In [2]:
async def jpt_ntb_extract_message_reactions(tlg_group_id: int, ids: List[int])->pd.DataFrame:
    loop = asyncio.get_event_loop()
    task = loop.create_task(extract_message_reactions(tlg_group_id, ids, chunk_size=200))
    result = await task
    # print("Result:", result)
    return result

In [3]:
df = telega_dump_to_pandas(dump_path=dump_path)


In [8]:
ids = df['id'].to_list()
# df_reactions = asyncio.run(jpt_ntb_extract_message_reactions(tlg_group_id, ids[141300:142000]))
df_reactions = asyncio.run(get_reactions(tlg_group_id, ids[141000:142000]))


INFO:pyrogram.connection.connection:Connecting...
INFO:pyrogram.connection.connection:Connected! Production DC2 - IPv4
INFO:pyrogram.session.session:NetworkTask started
INFO:pyrogram.session.session:Session initialized: Layer 158
INFO:pyrogram.session.session:Device: CPython 3.12.2 - Pyrogram 2.0.106
INFO:pyrogram.session.session:System: Windows 10 (en)
INFO:pyrogram.session.session:Session started
INFO:pyrogram.session.session:PingTask started
INFO:pyrogram.dispatcher:Started 12 HandlerTasks
INFO:root:retrieving reactions for 1000 messages
INFO:root:got reactions for 25 messages at 19:18:24.557303
INFO:pyrogram.dispatcher:Stopped 12 HandlerTasks
INFO:pyrogram.session.session:PingTask stopped
INFO:pyrogram.session.session:NetworkTask stopped
INFO:pyrogram.connection.connection:Disconnected
INFO:pyrogram.session.session:Session stopped


[148277, 148297, 148297, 148300, 148301, 148301, 148305, 148306, 148310, 148312, 148316, 148323, 148324, 148325, 148331, 148333, 148334, 148335, 148335, 148343, 148343, 148345, 148346, 148352, 148355, 148356, 148358, 148359]


In [10]:
len(df_reactions)

28

In [7]:
#149230 in ids[141900:142000] # df_reactions['msg_id'].to_list()

# 149230 in 
len(df_reactions) # ['msg_id'].to_list()


32

In [ ]:
df.to_parquet('df.parquet.gzip', compression='gzip')

In [ ]:
import tiktoken

# Load the cl100k_base tokenizer which is designed to work with the ada-002 model
tokenizer = tiktoken.get_encoding("cl100k_base")


# Tokenize the text and save the number of tokens to a new column
df['n_tokens'] = df.mess_text.apply(lambda x: len(tokenizer.encode(x)))



In [ ]:
# df.dtypes
# Visualize the distribution of the number of tokens per row using a histogram
df.n_tokens.hist(bins=200)
# df.tail(50)

